In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
data=pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')
data.head()

In [ ]:
print('We have {} rows and {} columns'.format(data.shape[0],data.shape[1]))

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

1. Alot of columns -except Outmome columns- has "0" value,which are incorrect data,because it impossibe to be the {Glucose,BloodPressure,SkinThickness,Insulin ,BMI,DiabetesPedigreeFunction and Age} to be 0.So it is possible that these values have been changed or filled with these values manually. We will change zero cells to be empty values.
Outcome column has real value which are {0,1},so it will not changed
  

In [ ]:
data=data.copy()
data.iloc[:,:-1]=data.replace(0,np.NaN)
data.head()

In [ ]:
data.isnull().sum()

As seen above ,after change zeros,we have got the true number of miessed values

We will fill the missed values with (mean) for every column,but we will divide each column into two parts.The first part for (negative cases) and the other part for the (positive cases), and get the (mean) for every part alone.

In [ ]:
Positives=data[data['Outcome']==1]
Negatives=data[data['Outcome']==0]

In [ ]:
Positives.shape

In [ ]:
Negatives.shape

In [ ]:
print('The numbet of diabetic is {} humans and number of healthy is {} humans'
      .format(Positives.shape[0],Negatives.shape[0]))

In [ ]:
column=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age']
for col in column:
    Positives[col].fillna(Positives[col].mean(),inplace = True)
Positives.head()

In [ ]:
column=['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age']
for col in column:
    Negatives[col].fillna(Negatives[col].mean(),inplace = True)
Negatives.head()

Now ,after filling the missing values for every part,we concat the two parts ,to be one part,as we get the data set 

In [ ]:
data=pd.concat([Negatives,Positives])
data.sample(5)

In [ ]:
plt.figure(figsize=(3,6))
sns.countplot(data['Outcome'])
plt.show()

In [ ]:
plt.figure(figsize=(5,5))
plt.pie(data['Outcome'].value_counts(),labels=(' negatives', ' positives'),
        explode = [0.03,0.03],autopct ='%1.1f%%'
        ,shadow = True, startangle = 270,
        labeldistance = 1.2, pctdistance =0.5)
plt.axis('equal')
plt.show()


In [ ]:
ndata = data.hist(figsize = (12,12))

In [ ]:
sns.pairplot(data=data)

In [ ]:
p=sns.pairplot(data, hue = 'Outcome')

In [ ]:
plt.figure(figsize=(9,5))
sns.heatmap(data.corr().sort_values(by=['Outcome'], ascending=False),annot=True)

As we see (Glucode,Insulin,BMI and SkinThickness) ,are most correaltion to the "Ootcome

In [ ]:
sns.distplot(data['Glucose'])

In [ ]:
sns.distplot(data['Insulin'])

In [ ]:
sns.distplot(data['BMI'])

In [ ]:
sns.distplot(data['SkinThickness'])

In [ ]:
sns.regplot(x='Glucose', y= 'Insulin' ,data=data)

In [ ]:
plt.subplots(figsize=(15,6))
sns.boxplot(x='Age', y='Glucose', data=data)

In [ ]:
sns.kdeplot(data['Age'],data['BloodPressure'],shade=True,cmap="Reds", shade_lowest=False)

In [ ]:
plt.figure(figsize=(8,7))
sns.boxenplot(y=data["Glucose"])
plt.show()

In [ ]:
plt.figure(figsize=(19,6))
sns.pointplot(data['Glucose'], data['Insulin'])
plt.show()

In [ ]:
X=data.iloc[:,:-1]
y=data.iloc[:,-1]

In [ ]:
scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
X = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=44, shuffle =True)

In [ ]:
GBRModel = GradientBoostingRegressor(n_estimators=100,max_depth=2,learning_rate = 1.5 ,random_state=33)
GBRModel.fit(X_train, y_train)
print('GBRModel Train Score is : ' , GBRModel.score(X_train, y_train))
print('GBRModel Test Score is : ' , GBRModel.score(X_test, y_test))
y_pred = GBRModel.predict(X_test)

In [ ]:
SVRModel = SVR(C = 1.0 ,epsilon=0.1,kernel = 'rbf') # it also can be : linear, poly, rbf, sigmoid, precomputed
SVRModel.fit(X_train, y_train)
print('SVRModel Train Score is : ' , SVRModel.score(X_train, y_train))
print('SVRModel Test Score is : ' , SVRModel.score(X_test, y_test))
y_pred = SVRModel.predict(X_test)

In [ ]:
DecisionTree = DecisionTreeRegressor( max_depth=3,random_state=33)
DecisionTree.fit(X_train, y_train)
print('DecisionTreeRegressor Train Score is : ' , DecisionTree.score(X_train, y_train))
print('DecisionTreeRegressor Test Score is : ' , DecisionTree.score(X_test, y_test))
y_pred = DecisionTree.predict(X_test)

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(1000, 300, 300), solver='adam', shuffle=False, tol = 0.0001)
mlp.fit(X_train, y_train)
print('DecisionTreeRegressor Train Score is : ' , mlp.score(X_train, y_train))
print('DecisionTreeRegressor Test Score is : ' , mlp.score(X_test, y_test))
y_pred = mlp.predict(X_test)

In [ ]:
accuracies = {'Model': ['GBRModel', 'SVRModel', 'DecisionTree',  'MLP'],
     'accuracy' : [GBRModel.score(X_test, y_test), SVRModel.score(X_test, y_test), DecisionTree.score(X_test, y_test),  mlp.score(X_test, y_test)]}

result = pd.DataFrame(data = accuracies)
result

In [ ]:
result.plot(x='Model', y='accuracy', kind='bar', figsize=(8, 8), title='Diabetes Prediction Accuracy', 
               sort_columns=True)